In [247]:
import pymysql, requests, re, time, json
import numpy as np, pandas as pd

## Mysql Template

import pymysql
db = pymysql.connect(host='119.29.190.115', user='root', password='njuacmicpc',
                         port=3306, db='weather', write_timeout = 6, read_timeout = 6)
cursor = db.cursor()

def sql_insert(db, cursor, table, data):
    keys = ', '.join(data.keys())
    values = ', '.join(['%s'] * len(data))
    sql = 'insert into {table}({keys}) values ({values}) on duplicate key update'.format(table=table, 
            keys=keys, values=values)
    update = ','.join([" {key} = %s".format(key=key) for key in data])
    sql += update
    #print (sql)
    try:
        if cursor.execute(sql, tuple(data.values())*2):
            print('Successful')
            db.commit()
        else:
            print ('Nothing to do')
    except:
        print ('Failed')
        db.rollback()

def get_list(data, i = 0, K = 2):
    n = len(data)
    return ','.join([str(x) for x in data[i * n // K: (i + 1) * n // K]])

url = 'http://api.data.cma.cn:8090/api'
form = {
    'userId': '541580345135c63Om',
    'pwd': 'vUXahFO',
    'dataFormat': 'json',
    'interfaceId': 'getSurfEleByTimeRangeAndStaID',
    'dataCode': 'SURF_CHN_MUL_HOR',
    'timeRange': '[{}0000,{}0000]'.format(time.strftime('%Y%m%d22', time.localtime(time.time() - 6 * 24 * 60 * 60)),
                 time.strftime('%Y%m%d22', time.localtime(time.time()))),
    'staIDs': '',
    'elements': 'TEM,Station_Id_C,Year,Mon,Day,Hour',
}

In [248]:
id_list = [54511,54527,53691,53693,53772,50431,53463,54342,54662,54161,50136,50953,58367,58027,58238,58245,58349,58457,58659,58321,58334,58847,59134,57993,58606,54823,54857,53982,57083,57297,57278,57447,57461,57494,59287,59493,59658,57687,57972,57957,59046,59431,59758,56187,57204,57516,56778,56969,57816,57127]
id_list = [str(x) for x in id_list]
K = 2

station_df = pd.read_excel('./China_SURF_Station.xlsx')
station_dict = {}
for id, name, prov in zip(station_df.区站号, station_df.站名, station_df.省份):
    station_dict[str(id)] = (name, prov)

In [249]:
texts = []
for i in range(K):
    form['staIDs'] = get_list(id_list, i)
    response = requests.get(url = url, params = form)
    texts.append(response.text)
    
result = [json.loads(x) for x in texts]
for i in range(K):
    result[i] = result[i]['DS']

In [250]:
for i in range(K):
    for j in range(len(result[i])):
        name, prov = station_dict[result[i][j]['Station_Id_C']]
        result[i][j]['name'] = name
        result[i][j]['prov'] = prov
    result[i] = [ x for x in result[i] if x['Hour'] in ['2','8','14','20']]
    results = []
for i in range(K):
    results += result[i]

In [251]:
from collections import defaultdict
dic_by_id = defaultdict(lambda: list())
for x in results:
    dic_by_id[x['Station_Id_C']].append(x)

In [252]:
dic_by_name_agg = {}
for id in id_list:
    name = station_dict[id][0]
    cur = dic_by_id[id]
    aggregate_weather = []
    n = len(cur)
    i = 0; j = 0;
    while (cur[i]['Hour'] != '2'): i += 1
    while (i + 3 < n):
        avg = 0.25 * (float(cur[i]['TEM']) + float(cur[i+1]['TEM']) + float(cur[i+2]['TEM']) + float(cur[i+3]['TEM']))
        if(cur[i]['Day']!=cur[i+3]['Day']):
            print(name, id)
            print(cur[i]['Day'],cur[i+1]['Day'],cur[i+2]['Day'],cur[i+3]['Day'])
            print(cur[i]['Hour'],cur[i+1]['Hour'],cur[i+2]['Hour'],cur[i+3]['Hour'])
            while (cur[i]['Day']==cur[i+1]['Day']): i += 1
            continue
        else:
            aggregate_weather.append({'prov':cur[i]['prov'],'year':cur[i]['Year'],'month':cur[i]['Mon'],
                                  'day':cur[i]['Day'],'TEM_Avg':avg})
        i += 4
    dic_by_name_agg[name] = aggregate_weather

In [253]:
for key, dic_list in dic_by_name_agg.items():
    for dic in dic_list:
        cur = {}
        cur['name'] = key
        cur.update(dic)
        print (cur)
        sql_insert(db, cursor, 'TEM', cur)

{'name': '北京', 'prov': '北京', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 9.875}
Nothing to do
{'name': '北京', 'prov': '北京', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 11.849999999999998}
Nothing to do
{'name': '北京', 'prov': '北京', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 7.275}
Nothing to do
{'name': '北京', 'prov': '北京', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 7.625}
Nothing to do
{'name': '北京', 'prov': '北京', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 5.0}
Nothing to do
{'name': '西青', 'prov': '天津', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 12.325000000000001}
Nothing to do
{'name': '西青', 'prov': '天津', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 13.950000000000001}
Nothing to do
{'name': '西青', 'prov': '天津', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 8.200000000000001}
Nothing to do
{'name': '西青', 'prov': '天津', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 7.2}
Nothing to do
{'name': '西青', 'p

Nothing to do
{'name': '扬州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 16.200000000000003}
Nothing to do
{'name': '扬州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 19.525}
Nothing to do
{'name': '扬州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 16.175}
Nothing to do
{'name': '扬州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 13.65}
Nothing to do
{'name': '苏州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 16.625}
Nothing to do
{'name': '苏州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 17.075}
Nothing to do
{'name': '苏州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 19.3}
Nothing to do
{'name': '苏州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 17.575}
Nothing to do
{'name': '苏州', 'prov': '江苏', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 16.75}
Nothing to do
{'name': '杭州', 'prov': '浙江', 'yea

Nothing to do
{'name': '恩施', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 14.725}
Nothing to do
{'name': '恩施', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 12.175}
Nothing to do
{'name': '恩施', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 12.850000000000001}
Nothing to do
{'name': '恩施', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 11.975}
Nothing to do
{'name': '宜昌', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 16.6}
Nothing to do
{'name': '宜昌', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 16.275000000000002}
Nothing to do
{'name': '宜昌', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 16.45}
Nothing to do
{'name': '宜昌', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 10.825000000000001}
Nothing to do
{'name': '宜昌', 'prov': '湖北', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 8.425}
Nothing to do
{'name': 

Nothing to do
{'name': '昆明', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 14.575}
Nothing to do
{'name': '昆明', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 14.424999999999999}
Nothing to do
{'name': '昆明', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 14.825}
Nothing to do
{'name': '勐腊', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 17.75}
Nothing to do
{'name': '勐腊', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '3', 'TEM_Avg': 18.575}
Nothing to do
{'name': '勐腊', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '4', 'TEM_Avg': 19.25}
Nothing to do
{'name': '勐腊', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '5', 'TEM_Avg': 19.925}
Nothing to do
{'name': '勐腊', 'prov': '云南', 'year': '2018', 'month': '11', 'day': '6', 'TEM_Avg': 20.0}
Nothing to do
{'name': '贵阳', 'prov': '贵州', 'year': '2018', 'month': '11', 'day': '2', 'TEM_Avg': 13.05}
Nothing to do
{'name': '贵阳', 'prov': '贵州', 'year

In [269]:
with open("./id_list", "w") as fp:
    for id in id_list:
        fp.write('{}\n'.format(id))

In [270]:
with open("./id_list", "r") as fp:
    for x in fp:
        print (x[:-1])

54511
54527
53691
53693
53772
50431
53463
54342
54662
54161
50136
50953
58367
58027
58238
58245
58349
58457
58659
58321
58334
58847
59134
57993
58606
54823
54857
53982
57083
57297
57278
57447
57461
57494
59287
59493
59658
57687
57972
57957
59046
59431
59758
56187
57204
57516
56778
56969
57816
57127
